<a href="https://colab.research.google.com/github/jacomyma/mapping-controversies/blob/main/notebooks/Words_and_documents_with_text_to_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🍇 Words and documents with text to network

**Inputs:**
* a list of documents with their text content (CSV)
* a list of words (CSV)

**Output:** a bipartite network of documents and words (GEXF)

This script tells you which words are in which documents. Words can be expressions (e.g., named entities).

## How to use

1. Put your input files in the same folder as the notebook
1. Edit the settings if needed. CHECK THE COLUMN NAMES!
1. Run all the cells
1. Take the output file from the notebook folder

# SETTINGS

In [ ]:
# Input file 1: documents
input_file_documents = "documents.csv"
# Which column contains the text?
documents_text_column = "Text"
# Which column contains the document name or ID?
documents_id_column = "Article"

# Input file 2: small list of words
input_file_words = "words.csv"
# Which column contains the words?
words_text_column = "text"

# Delete documents that contain none of the words?
discard_unrelated_documents = True

# Output file
output_file_network = "terms-document-network.gexf"


# SCRIPT

### Install and import libraries
This notebook draws on existing code.
You can ignore the output.

In [ ]:
# Install (if needed)
!pip install pandas
!pip install spacy
!pip install networkx

# Import
import csv
import pandas as pd
import networkx as nx

print("Done.")

Done.


### Read the input file 1 (documents)

In [ ]:
doc_df = pd.read_csv(input_file_documents, quotechar='"', encoding='utf8', doublequote=True, quoting=csv.QUOTE_NONNUMERIC, dtype=object)
print("Preview of the document list:")
doc_df

Preview of the document list:


,Article,Text
0,Search engine privacy,search engine privacy is a subset of internet ...
1,Member Berries,"""member berries"" is the first episode in the t..."
2,Real-name system,a real-name system is a system in which users ...
3,CSipSimple,csipsimple is a voice over internet protocol (...
4,Spam blog,"a spam blog, also known as an auto blog or the..."
5,Worldwide Protests for Free Expression in Bang...,the worldwide protests for free expression in ...
6,FTC fair information practice,the united states federal trade commission's ...
7,Spam mass,"spam mass is defined as ""the measure of the im..."


### Read the input file 2 (words)

In [ ]:
word_df = pd.read_csv(input_file_words, quotechar='"', encoding='utf8', doublequote=True, quoting=csv.QUOTE_NONNUMERIC, dtype=object)
print("Preview of the word list:")
word_df

Preview of the word list:


,text,type,count-occurences-total,count-documents
0,aol,ORG,7,1
1,yahoo,ORG,19,2
2,google,ORG,28,1
3,2008,DATE,4,2
4,third,ORDINAL,5,3
...,...,...,...,...
271,spam mass,PERSON,1,1
272,zoltán gyöngyi,ORG,1,1
273,hector garcia-molina,PERSON,1,1
274,stanford university in association,ORG,1,1


### Wrangle the data

In [ ]:
# Get a set of the words
words = set()
for index, row in word_df.iterrows():
  words.add(row[words_text_column])

# Init data for output
network_doc_set = set()
network_word_set = set()
network_edge_list = []

# Search words in documents
for index, row in doc_df.iterrows():
  text = row[documents_text_column].lower()
  count_per_word = {}
  flag = False
  for word in words:
    count = text.count(word.lower())
    count_per_word[word] = count
    if count > 0:
      flag = True

  if flag or not discard_unrelated_documents:
    doc_id = row[documents_id_column]
    network_doc_set.add(doc_id)
    for word in words:
      count = count_per_word[word]
      if count > 0:
        network_word_set.add(word)
        network_edge_list.append((doc_id,word,{"count":count}))



### Make and save network

In [ ]:
# Build the nodes
nodes = []
doc_df_no_text = doc_df.drop(columns=[documents_text_column]) 
for index, row in doc_df_no_text.iterrows():
  if row[documents_id_column] in network_doc_set:
    nodes.append((row[documents_id_column], {**row, 'label':row[documents_id_column], 'type':'document'}))

for index, row in word_df.iterrows():
  if row[words_text_column] in network_word_set:
    nodes.append((row[words_text_column], {**row, 'label':row[words_text_column], 'type':'term'}))

# Build edges
edges = network_edge_list

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
nx.write_gexf(G, output_file_network)
print("Done.")

Done.
